# Mortality Rates in the Netherlands

> Last updated April 18, 2020

Source data:
- [Weekly mortatlity figures](https://opendata.cbs.nl/statline/portal.html?_la=nl&_catalog=CBS&tableId=70895ned&_theme=75)
- [Montly population figures](https://opendata.cbs.nl/statline/portal.html?_la=nl&_catalog=CBS&tableId=37943ned&_theme=62)
- [Daily COVID-19 deaths](https://www.rivm.nl/coronavirus-covid-19/grafieken)

Entry for mortality week 15 2020 added manually based on latest reports, which was not reflected in source data at time of writing ([see here](https://www.cbs.nl/en-gb/news/2020/16/mortality-in-second-week-of-april-estimated-at-5-000)).



In [ ]:
import pandas as pd

In [ ]:
deceased = pd.read_csv(
    'deceaseddata20200418.csv',
    sep=';',
    names=['id', 'sex', 'age', 'per', 'd'],
    skiprows=1,  # skip header
    dtype={'id':int,'sex':int,'age':int,'per':str,'d':int}
)

In [ ]:
period_filters = (
    ~deceased.per.str.contains('X0') &  # remove incomplete first weeks of year
    ~deceased.per.str.contains('W101') &  # remove first weeks of year that could start in prev year
    ~deceased.per.str.contains('W152') &  # remove incomplete last week of year
    ~deceased.per.str.contains('W153') &  # remove incomplete last week of year (in 2011)
    ~deceased.per.str.contains('JJ')  # remove full year data contained in series
)

In [ ]:
deceased = deceased[period_filters]

In [ ]:
deceased['dt'] = pd.to_datetime(deceased.per + str(1), format='%GW1%V%u')

In [ ]:
# confirm the only date format that remains is in W-format (weeks)
sum(deceased.per.str.contains('W')) == len(deceased.per.str.contains('W'))

In [ ]:
# confirm that the inferred year matches the year in the period column
[int(year) for year in deceased.per.str[:4].tolist()] == pd.DatetimeIndex(deceased['dt']).year.tolist()

In [ ]:
# keep only broad demographics (all genders and ages)
all_genders = deceased['sex']==1100
all_ages = deceased['age']==10000

demo_filters = all_genders & all_ages

In [ ]:
deceased = deceased[demo_filters]

In [ ]:
# cleanup columns
deceased = deceased.drop(['id','sex','age','per'], axis=1)

In [ ]:
# quick plot show no spikes other than expected yearly seasonality
deceased.plot(x='dt')

In [ ]:
pop = pd.read_csv(
    'populationdata20200418.csv',
    sep=';',
    usecols=[1, 9],
    names=['per', 'pop'],
    skiprows=1,  # skip header
    dtype={'per':str,'pop':int}
)

In [ ]:
year_filters = (
    ~pop.per.str.contains('JJ') &
    ~pop.per.str.contains('KW')
)

In [ ]:
pop = pop[year_filters]

In [ ]:
pop['dt'] = pd.to_datetime(pop.per, format='%YMM%m')

In [ ]:
pop = pop.drop('per', axis=1)

In [ ]:
# quick plot to confirm data - population grew from roughly 15 to >17 million in time period
pop.plot(x='dt')

In [ ]:
pop.head()

In [ ]:
# merge on month grouper (population data is not available at weekly granularity)
merged = pd.merge(
    deceased.assign(grouper=deceased['dt'].dt.to_period('M')),
    pop.assign(grouper=pop['dt'].dt.to_period('M')),
    how='left',
    on='grouper'
)

In [ ]:
# cleanup columns
merged = merged.drop(['grouper', 'dt_y'], axis=1).fillna(method='ffill')

In [ ]:
# compute mortality rate (typically reported as deaths per 1000)
merged['mrate'] = merged['d'] / merged['pop'] * 1000

In [ ]:
# quick plot to confirm data is properly scaled
merged.plot(x='dt_x', y='mrate')

In [ ]:
# last three available weeks
merged[-3:]

In [ ]:
# create week and year columns for plot data
merged['week'] = merged.dt_x.dt.week
merged['year'] = merged.dt_x.dt.year

In [ ]:
plotdata = merged.pivot(index='week',columns='year',values='mrate')

In [ ]:
# convert to string headers as integers break pandas/bokeh
plotdata.columns = [str(header) for header in plotdata.columns.tolist()]

In [ ]:
plotdata['mn'] = plotdata.iloc[:,:-1].mean(axis=1)  # calculate means for 1995-2019

In [ ]:
plotdata['s'] = plotdata.iloc[:,:-2].std(axis=1)  # calculate standard deviations for 1995-2019

In [ ]:
plotdata['upper'] = plotdata['mn'] + 2 * plotdata['s']  # upper 2-sigma CI level

In [ ]:
plotdata['lower'] = plotdata['mn'] - 2 * plotdata['s']  # lower 2-sigma CI level

In [ ]:
plotdata = plotdata.drop('s', axis=1)  # remove standard deviation from plotdata

In [ ]:
# sanity check
plotdata.head(3)

In [ ]:
cd = pd.read_csv(
    'coronadeaths20200418.csv',
    sep=';',
    names=['date', 'new', 'known'],
    skiprows=1,
    dtype={'date':str,'new':int,'known':int}
)

In [ ]:
cd.date = cd.date + ' 2020'

In [ ]:
for i in range(len(cd.date.tolist())):
    if cd.date[i][1] == ' ':
        cd.date[i] = '0' + str(cd.date[i])
    cd.date[i] = str(cd.date[i]).replace('feb', 'February')
    cd.date[i] = str(cd.date[i]).replace('mrt', 'March')
    cd.date[i] = str(cd.date[i]).replace('apr', 'April')

In [ ]:
cd.index = pd.to_datetime(cd.date, format='%d %B %Y')

In [ ]:
cdw = cd.resample('W').sum()

In [ ]:
cdw['total'] = cdw.new + cdw.known

In [ ]:
cdw

In [ ]:
cdw.index = [i.isocalendar()[1] for i in cdw.index]
cdw.index.name = 'week'

In [ ]:
cdw

In [ ]:
for i in cdw.index[1:7]:
    plotdata.loc[i, 'coronadeaths'] = (cdw.loc[i, 'total'] / merged.loc[1262, 'pop'] * 1000) + plotdata.loc[i, 'mn']

In [ ]:
plotdata.loc[15]

In [ ]:
plotdata.head(25)

In [ ]:
from bokeh.plotting import figure, output_file, save
from bokeh.models import Span, Label, Title

In [ ]:
output_file('mortality.html', title='Mortality Rate in the Netherlands')

cols=len(plotdata.columns)

fig = figure(
    tools='pan,wheel_zoom,box_zoom,save,reset',
    x_axis_label='Week Number',
    x_minor_ticks=10,
    x_range=[1,52],
    y_range=[0.1,0.3],
    y_axis_label='Mortality Rate (per 1000 citizens)',
    plot_width=1000,
    plot_height=600,
    toolbar_location='above'
)

fig.title.text = 'Weekly Mortality Rate in the Netherlands 1995-2020'
fig.title.align = 'center'
fig.title.text_font_size = '25px'

fig.add_layout(
    Title(
        text='Source: Statistics Netherlands (2020), RIVM (2020). Preliminary data for 2019 and 2020.', 
        align='left',
        text_font_size='12px'
    ),
    'below'
)


fig.add_layout(Span(
    location=9,
    dimension='height',
    line_color='red',
    line_dash='dashed',
    line_width=1.5,
    line_alpha=0.5
))

fig.add_layout(Label(
    x=1.5,
    y=0.261,
    text='First COVID-19 death',
    render_mode='css',
    text_font_size='14px'
))

for i in range(cols-6):  # add all historic years
    fig.line(
        x=plotdata.index,
        y=plotdata.iloc[:,i],
        line_color='grey',
        line_alpha=0.15,
        line_width=1
    )

fig.line(
    x=plotdata.index,
    y=plotdata['2019'],
    line_color='grey',
    line_alpha=0.15,
    line_width=1
)


fig.line(
    x=plotdata.index,
    y=plotdata.mn,
    line_color='black',
    line_alpha=0.5,
    line_width=1.5,
    legend_label='Average (1995-2019)'
)

fig.line(
    x=plotdata.index,
    y=plotdata.upper,
    line_color='black',
    line_alpha=0.5,
    line_width=1.5,
    line_dash='dotted',
    legend_label='95% CI (1995-2019)'
)

fig.line(
    x=plotdata.index,
    y=plotdata.lower,
    line_color='black',
    line_alpha=0.5,
    line_width=1.5,
    line_dash='dotted'
)


fig.line(
    x=plotdata.index,
    y=plotdata['2018'],
    line_color='blue',
    line_alpha=1,
    line_width=1.5,
    legend_label='2018 (influenza epidemic)'
)

fig.varea(
    x=plotdata.index[8:],
    y1=plotdata['coronadeaths'][8:],
    y2=plotdata['2020'][8:],
    fill_alpha=0.15,
    fill_color='grey'
)

fig.line(
    x=plotdata.index,
    y=plotdata['2020'],
    line_color='red',
    line_alpha=1,
    line_width=3,
    legend_label='2020 (COVID-19 outbreak)'
)

fig.line(
    x=plotdata.index,
    y=plotdata['coronadeaths'],
    line_color='orange',
    line_alpha=1,
    line_width=3,
    line_dash='dashed',
    legend_label='Avg + reported COVID-19 deaths (RIVM)'
)

fig.add_layout(Label(
    x=16,
    y=0.26,
    text='Potential under-reported cases',
    render_mode='css',
    text_font_size='14px'
))

save(fig)